In [84]:
import math

import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import functions as fn
from IPython.display import display, Math
import scipy.integrate as int


"""
Definicion de los operadores L2 , Lx,Ly,Lz en coordenadas esfericas Para ello usaremos Sympy para definir estos operadores

"""
#Definimos parametros a usar en unidades atómicas
hbar = 1
me = 1

w, phi, theta  = sp.symbols('w phi theta', real=True)
f = sp.Function('Y_{lm}')(phi,theta)
hb = sp.symbols('hbar')
L,M = sp.symbols('l m', integer=True)

# Comencemos con uno sencillo, operador Lz
def Lz_op (f):
    Lz = -sp.I*hb*sp.diff(f,phi)
    return Lz

display (Math(r"\hat{L_{z}} =" + sp.latex(Lz_op(f))))

# Ahora sigamos con el operador L2

def L2_op (f):
    L2 = -hb**2*(sp.diff(f,theta,2)   +  sp.cot(theta)*sp.diff(f,theta)+1/(sp.sin(theta))**2 *sp.diff(f,phi,2))
    return L2

display (Math(r"\hat{L_{2}} =" + sp.latex(L2_op(f))))

# y terminamos con los operadores Lx y Ly

def Lx_op (f):
    Lx = sp.I*hb*(-sp.sin(phi)*sp.diff(f,theta) + sp.cot(theta)*sp.cos(phi)*sp.diff(f,phi))
    return Lx

def Ly_op (f):
    Ly = -sp.I*hb*(sp.cos(phi)*sp.diff(f,theta)   -  sp.cot(theta)*sp.sin(phi)*sp.diff(f,phi))
    return Ly

display (Math(r"\hat{L_{x}} =" + sp.latex(Lx_op(f))))
display (Math(r"\hat{L_{y}} =" + sp.latex(Ly_op(f))))

# Ahora vamos a comprobar su funcionalidad aplicando sobre ellos los armonicos esfericos.

def eferic_armonics_symbol(l,m):
    ef = (sp.sqrt((2*l+1)/(4*sp.pi) * sp.factorial(l+abs(m))/sp.factorial(l-abs(m)) )
          * sp.exp(sp.I * m * phi) * 1/(2**l * sp.factorial(l)) *
          (1-w**2)**(abs(m)/2)* sp.Derivative((w**2-1)**l,(w,l+abs(m))))
    ef = ef.subs(w,sp.cos(theta))
    return ef

def eferic_armonics_numeric(l,m):
    ef = (sp.sqrt((2*l+1)/(4*sp.pi) * sp.factorial(l+abs(m))/sp.factorial(l-abs(m)) )
          * sp.exp(sp.I * m * phi) * 1/(2**l * sp.factorial(l)) *
          (1-w**2)**(abs(m)/2)* sp.Derivative((w**2-1)**l,(w,l+abs(m))))
    ef = ef.subs(w,sp.cos(theta))
    return ef

def integrate_2d(Z, x, y, method):
    """
    Z: 2D array shaped (len(y), len(x))  -> f(y_i, x_j)
    x: 1D array (len Nx)
    y: 1D array (len Ny)
    method: 'rect', 'trapz', or 'simps'
    """
    if method == 'rect':
        dx = x[1]-x[0]
        dy = y[1]-y[0]
        return np.sum(Z) * dx * dy
    elif method == 'trapz':
        return np.trapezoid(np.trapezoid(Z, x=x, axis=1), x=y, axis=0)
    elif method == 'simps':
        return int.simpson(int.simpson(Z, x=x, axis=1), x=y, axis=0)
    else:
        raise ValueError("method must be 'rect','trapz' or 'simps'")


display (Math(r"Y_{lm}(\theta,\phi) =" + sp.latex(eferic_armonics_symbol(L,M))))

# Veamos que se cumple la ortogonalidad de estas funciones
thetvals,phivals = np.linspace(0,np.pi,500) , np.linspace(0,2*np.pi,500)
Tvals, Pvals = np.meshgrid(thetvals, phivals)

for l in range(5):
    for m in range(-l,l+1):
        eferic_num = sp.lambdify((theta,phi), eferic_armonics_numeric(l,m))
        eferic_num = eferic_num(Tvals,Pvals)
        display(Math(f"Y_{{{l}{m}}}" +r"(\theta,\phi) = " +
                     sp.latex( integrate_2d(np.conjugate(eferic_num)*eferic_num,thetvals,phivals,'trapz')      )))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

IndexError: list assignment index out of range